# **Phishing Website Dataset Construction**





## **Acknowledgement**

This Google Colab notebook was originally created by Github user shreyagopal and can be found [here](https://colab.research.google.com/github/shreyagopal/Phishing-Website-Detection-by-Machine-Learning-Techniques/blob/master/URL%20Feature%20Extraction.ipynb). The original version was developed in order to extract features from URLs that can be used to determine whetehr or not it's a phishing link or not.

## **Modifications for New Project**

This notebook has been adapted and modified by `Inukai, Masayuki` and `Saquilayan, Arturo Miguel` for use in their `Machine Learning Final Project`.

The modifications include:
1. Changing the dataset where the Phishing and Legitimate URLs were acquired
2. Modifying the formatting and content of the Python Notebook
3. Removing Web Traffic feature from the list of columns of the reslulting .csv file
4. Modifying `getDomain` function to accommodate links without `http` or `https` or `www` prefixes
5. Adding more link shortening services to the `tinyURL` function

#**1. Importing Important Packages**

In [174]:
import pandas as pd
from google.colab import drive


# **2. Data Preprocessing**

## **2.1. Loading the Dataset**

The URLs used in this notebook were acquired from this [Kaggle dataset](https://www.kaggle.com/datasets/taruntiwarihp/phishing-site-urls/data).

First, Google Drive must be mounted.

In [175]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Then, path must be changed to the right folder

In [176]:
%cd /content/drive/Shared drives/CS 180 2324B Final Project/datasets

/content/drive/Shared drives/CS 180 2324B Final Project/datasets


Folder must have these two .csv files: `preprocessed_dataset.csv` and `shorturl_services_list.csv`

In [177]:
%ls

preprocessed_dataset.csv  shorturl_services_list.csv


The command below loads `test_dataset.csv` into this notebook.

In [178]:
#loading the phishing URLs data to dataframe
urls = pd.read_csv("preprocessed_dataset.csv")
urls.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


## **2.2. Extracting good and bad URLs**


From the command below, it's known that this dataset has `549346 rows` and `2 columns`.

In [179]:
urls.shape

(549346, 2)

The dataset must only have unique values. Hence, `drop_duplicates()` is used on the dataset. According to the dataset page on Kaggle, there should be `507196 unique rows`.

In [180]:
urls = urls.drop_duplicates()
urls.shape

(507196, 2)

Now that the dataset only contains unique values, `7500 good URLs` and `7500 bad URLs`will be randomly extracted from the dataset.

In [181]:
good_urls = urls[urls['Label'] == 'good']
good_urls = good_urls.sample(n = 7500, random_state = 18).copy()
good_urls = good_urls.reset_index(drop=True)
good_urls.shape

(7500, 2)

In [182]:
bad_urls = urls[urls['Label'] == 'bad']
bad_urls = bad_urls.sample(n = 7500, random_state = 9).copy()
bad_urls = bad_urls.reset_index(drop=True)
bad_urls.shape


(7500, 2)

# **3. Feature Extraction:**

In this step, features are extracted from the URLs dataset.

The extracted features are categorized into


1.   Address Bar based Features
2.   Domain based Features
3.   HTML & Javascript based Features



### **3.1. Address Bar Based Features:**

Many features can be extracted that can be consided as address bar base features. Out of them, below mentioned were considered for this project.


*   Domain of URL
*   IP Address in URL
*   "@" Symbol in URL
*   Length of URL
*   Depth of URL
*   Redirection "//" in URL
*   "http/https" in Domain name
*   Using URL Shortening Services “TinyURL”
*   Prefix or Suffix "-" in Domain

Each of these features are explained and the coded below:

In [183]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

#### **3.1.1. Domain of the URL**
Here, we are just extracting the domain present in the URL. This feature doesn't have much significance in the training. May even be dropped while training the model.

In [184]:
# 1.Domain of the URL (Domain)
def getDomain(url):
  if not re.match(r'^[a-zA-Z]+://', url):
    url = 'http://' + url
  domain = urlparse(url).netloc
  if domain.startswith("www."):
      domain = domain.replace("www.", "", 1)
  return domain

print(getDomain("www.microdynetech.com/wire.htm"))

microdynetech.com


#### **3.1.2. IP Address in the URL**

Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.

If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).



In [185]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip


#### **3.1.3. "@" Symbol in URL**

Checks for the presence of '@' symbol in the URL. Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

If the URL has '@' symbol, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [186]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1
  else:
    at = 0
  return at

#### **3.1.4. Length of URL**

Computes the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.

If the length of URL >= 54 , the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [187]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0 # Legitimate
  else:
    length = 1 # Phishing
  return length

#### **3.1.5. Depth of URL**

Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

The value of feature is a numerical based on the URL.

In [188]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

#### **3.1.6. Redirection "//" in URL**

Checks the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. The location of the “//” in URL is computed. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

If the "//" is anywhere in the URL apart from after the protocal, thee value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [189]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

#### **3.1.7. "http/https" in Domain name**

Checks for the presence of "http/https" in the domain part of the URL. The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.

If the URL has "http/https" in the domain part, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [190]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

#### **3.1.8. Using URL Shortening Services “TinyURL”**

URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

The list is acquired from Github user sambokai from their [repository](https://github.com/sambokai/ShortURL-Services-List/tree/master), which has a grand total of over 600 URL shortening services.

In [191]:
list_shortening_services = pd.read_csv("shorturl_services_list.csv")
list_shortening_services = list_shortening_services['domain name']
list_shortening_services.head()

0       0rz.tw
1    1-url.net
2       126.am
3     1link.in
4       1tk.us
Name: domain name, dtype: object

In [192]:
# Listing shortening services
parsed_list_shortening_services = []
for service in list_shortening_services:
  split_domain = service.split(".")
  link = split_domain[0] + "\." + split_domain[1]
  parsed_list_shortening_services.append(link)

# Combining each and every list shortening service via '|'
shortening_services = '|'.join(parsed_list_shortening_services)

# Converting into raw string
shortening_services = rf"""
{shortening_services}
"""


In [193]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

#### **3.1.9. Prefix or Suffix "-" in Domain**

Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [194]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

### **3.2. Domain Based Features:**

Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

*   DNS Record
*   Website Traffic
*   Age of Domain
*   End Period of Domain

Each of these features are explained and the coded below:

In [195]:
!pip install python-whois

In [196]:
# importing required packages for this section
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

#### **3.2.1. DNS Record**

For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname.
If the DNS record is empty or not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [197]:
# 11.DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

#### **3.2.2. Web Traffic**

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

If the rank of the domain < 100000, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [198]:
# 12.Web traffic (Web_Traffic)
def web_traffic(url):
  try:
      # Filling the whitespaces in the URL if any
      encoded_url = urllib.parse.quote_plus(url)
      alexa_url = f"http://data.alexa.com/data?cli=10&dat=s&url={encoded_url}"

      # Making the request using requests library
      response = requests.get(alexa_url)
      response.raise_for_status()  # Check for HTTP errors

      # Parsing the response with BeautifulSoup
      soup = BeautifulSoup(response.content, "xml")
      reach_tag = soup.find("REACH")

      if reach_tag and 'RANK' in reach_tag.attrs:
          rank = int(reach_tag['RANK'])
      else:
          return 1  # Return default rank if REACH tag or RANK attribute is missing

      return 1 if rank < 100000 else 0
  except (requests.RequestException, TypeError) as e:
      print(f"An error occurred: {e}")
      return 1  # Return default rank in case of an error

#### **3.2.3. Age of Domain**

This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but different between creation and expiration time.

If age of domain > 12 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [199]:
# 13.Survival time of domain: The difference between termination time and creation time (Domain_Age)
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

#### **3.2.4. End Period of Domain**

This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or less  for this project.

If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [200]:
# 14.End time of domain: The difference between termination time and current time (Domain_End)
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

## **3.3. HTML and JavaScript based Features**

Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

*   IFrame Redirection
*   Status Bar Customization
*   Disabling Right Click
*   Website Forwarding

Each of these features are explained and the coded below:

In [201]:
# importing required packages for this section
import requests

### **3.3.1. IFrame Redirection**

IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation.

If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [202]:
# 15. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

### **3.3.2. Status Bar Customization**

Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the “onMouseOver” event, and check if it makes any changes on the status bar

If the response is empty or onmouseover is found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [203]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response):
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

### **3.3.3. Disabling Right Click**

Phishers use JavaScript to disable the right-click function, so that users cannot view and save the webpage source code. This feature is treated exactly as “Using onMouseOver to hide the Link”. Nonetheless, for this feature, we will search for event “event.button==2” in the webpage source code and check if the right click is disabled.

If the response is empty or onmouseover is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).




In [204]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

### **3.3.4. Website Forwarding**
The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times.




In [205]:
# 18.Checks the number of forwardings (Web_Forwards)
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

## **4. Computing URL Features**

Create a list and a function that calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.

In [206]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))

  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  # features.append(web_traffic(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))

  # HTML & Javascript based features (4)
  try:
    response = requests.get(url)
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)

  return features

### **4.1. Legitimate URLs:**

Now, feature extraction is done on legitimate URLs.

In [207]:
good_urls.shape

(7500, 2)

In [208]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(len(good_urls)):
  url = good_urls['URL'].iloc[i]
  legi_features.append(featureExtraction(url,label))

In [209]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record',
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,ucmo.edu,0,0,0,4,0,0,0,0,0,1,1,1,1,1,1,0
1,amazon.com,0,0,0,4,0,0,0,0,0,1,1,1,1,1,1,0
2,juicyfinder.com,0,0,0,2,0,0,0,0,0,1,1,1,1,1,1,0
3,martindale.com,0,0,1,3,0,0,0,0,0,1,1,1,1,1,1,0
4,montrealladies.com,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,0


### **4.2. Phishing URLs:**

Now, feature extraction is performed on phishing URLs.

In [210]:
bad_urls.shape

(7500, 2)

In [211]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(len(bad_urls)):
  url = bad_urls['URL'].iloc[i]
  phish_features.append(featureExtraction(url,label))

In [212]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection',
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record',
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,udecodocs.net,0,0,0,3,0,0,0,0,0,1,1,1,1,1,1,1
1,shorelinegreenbusiness.com,0,0,1,4,0,0,0,0,0,1,1,1,1,1,1,1
2,callcenteronline.cl,0,0,0,4,0,0,0,0,0,1,1,1,1,1,1,1
3,timschool11.ru,0,0,0,5,0,0,0,0,0,1,1,1,1,1,1,1
4,sawko.net,0,0,0,3,0,0,0,0,0,1,1,1,1,1,1,1


## **5. Final Dataset**

In the above section, we formed two dataframes of legitimate & phishing URL features. Now, we will combine them into a single dataframe and export the data to a `.csv` file to create a `Decision Tree Model`.

In [213]:
#Concatenating the dataframes into one
final_dataset = pd.concat([legitimate, phishing]).reset_index(drop=True)
final_dataset.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,ucmo.edu,0,0,0,4,0,0,0,0,0,1,1,1,1,1,1,0
1,amazon.com,0,0,0,4,0,0,0,0,0,1,1,1,1,1,1,0
2,juicyfinder.com,0,0,0,2,0,0,0,0,0,1,1,1,1,1,1,0
3,martindale.com,0,0,1,3,0,0,0,0,0,1,1,1,1,1,1,0
4,montrealladies.com,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,0


In [214]:
final_dataset.tail()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
14995,us-battle-net.ccwow.asia,0,0,1,4,0,0,0,0,0,1,1,1,1,1,1,1
14996,linkzip.net,0,0,0,3,0,0,1,0,0,1,1,1,1,1,1,1
14997,smxqfps.biz,0,0,0,2,0,0,0,0,0,1,1,1,1,1,1,1
14998,s70.n57.n84.n66.static.myhostcenter.com,0,0,1,5,0,0,0,0,0,1,1,1,1,1,1,1
14999,klaretech.com,0,1,1,2,0,0,0,0,0,1,1,1,1,1,1,1


In [215]:
final_dataset.shape

(15000, 17)

In [216]:
# Storing the data in CSV file
final_dataset.to_csv('final_dataset.csv', index=False)

## **6. Conclusion**

With this, the dataset for testing has been formed. This notebook has extracted 16 features 2760 URLs, with 1380 of them being phishing URLs while the rest are legitimate.



## **7. References**

* https://archive.ics.uci.edu/ml/datasets/Phishing+Websites
* https://colab.research.google.com/github/shreyagopal/Phishing-Website-Detection-by-Machine-Learning-Techniques/blob/master/URL%20Feature%20Extraction.ipynb#scrollTo=mtwQiRotZ2GD
* https://github.com/sambokai/ShortURL-Services-List/tree/master

